In [ ]:
!pip install xgboost

In [ ]:
from pathlib import Path

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from functools import partial

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer, StandardScaler
from sklearn.tree import DecisionTreeRegressor

In [ ]:
from xgboost import XGBRegressor

In [ ]:
%matplotlib inline

## Helper functions

In [ ]:
def rmse(preds: np.array, y: np.array) -> float:
    return np.sqrt(np.mean((preds - y) ** 2))

In [ ]:
def evaluate_model(
    clf,
    X_train: np.array,
    y_train: np.array,
    X_test: np.array,
    y_test: np.array,
) -> None:
    print(f"train rmse: {rmse(clf.predict(X_train), y_train):.2f}")
    print(f"test rmse: {rmse(clf.predict(X_test), y_test):.2f}")

# 0. Data

We will be using the King County house sales dataset, documented here: https://rdrr.io/cran/moderndive/man/house_prices.html

Our task is to regress the sale price of houses given their other features.

In [ ]:
# Fetch data from S3 if it doesn't exist locally
if not Path('kc_house_data.csv').exists():
    !wget https://s3-eu-west-1.amazonaws.com/faculty-client-teaching-materials/bagging-and-boosting/kc_house_data.csv

In [ ]:
# Preprocessing

In [ ]:
df = pd.read_csv("kc_house_data.csv", index_col=0)

In [ ]:
df["date"] = pd.to_datetime(df["date"])

In [ ]:
func = lambda d: pd.Series(data=[d.year, d.month, d.day])

df[["year", "month", "day"]] = df["date"].apply(func)

In [ ]:
df = df.drop("date", axis="columns")

In [ ]:
# drop very large/ outlier house prices for ease of analysis.
df = df.loc[df["price"] <= 10 ** 6]

In [ ]:
X, y = df.drop("price", axis="columns"), df["price"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
evaluate = partial(
    evaluate_model,
    X_train=X_train,
    y_train=y_train,
    X_test=X_test,
    y_test=y_test,
)

# 1. Boosting by hand

Let's do some very simple boosting by hand. Below, you'll find code to train a simple linear model `lasso` (after some feature engineering) to predict house prices. The results aren't very good, but this is a sensible simple model to begin with- it's reasonable to assume that the price of a house should be monotonic in individual variables like the number of bedrooms/bathrooms and the square footage of the of property.
1. Train a simple `DecisionTreeRegressor` model, `dt_res`, with `max_depth=3` to predict the residual `y_train - lasso.predict(X_train)`.
2. Now combine the models `lasso` and `dt_res` into a class with `.fit` and `.predict` methods representing a single model whose output should be of the form `lasso(x) + dt_res(x)`. Note that the `.fit` method will have to proceed sequentially, first fitting the model `lasso` and then `dt_res` afterwards.
3. What is the `rmse` of the combined (boosted) model `lasso + dt_res` as compared to just `lasso` or a single `DecisionTreeRegressor` trained to directly predict `y_train` from `X`?
4. It's worth making sure that our improved results aren't because `DecisionTreeRegressor` and `Lasso` are somehow especially well suited to be combined together. Check this by repeating tasks `1-3`, but with the original `lasso` model replaced by a simple `DecisionTreeRegressor` (again with `max_depth=3`), so that you're stacking two decision tree models on top of each other. Feel free to experiment with other regression models or with a higher number of `DecisionTreeRegressor` instances as well!

In [ ]:
## Compute some extra features to help the lasso model learn more accurate monotonic relationships
# from the data.
def poly_features(X: np.array):
    return np.hstack([X, X ** 2, X ** 3])


lasso = Pipeline(
    [
        (
            "variable_transformation",
            FunctionTransformer(poly_features, validate=False),
        ),
        ("lasso_regressor", Lasso(normalize=True, max_iter=5000)),
    ]
)

In [ ]:
lasso.fit(X_train, y_train)

In [ ]:
evaluate(lasso)

In [ ]:
# your code here

In practice, boosting models are almost always implemented using decision trees as base estimators because of how fast they are to train, and thanks to their applicability in a wide variety of scenarios. You won't have to do this by hand, however! Basic implementations of boosting models using decision trees for classification and regression are made available in `sklearn.ensemble` via the classes `GradientBoostingRegressor` and `GradientBoostingClassifier`.

5. Check that a regressor of the form `GradientBoostingRegressor(n_estimators=2, max_depth=3, learning_rate=1)` (don't worry about what the learning rate is right now- we'll discuss this very shortly) gives results which are close to those of the hand-made boosting model consisting of two decision trees that you implemented. The results might not coincide exactly because of some implementation details for how decision trees are trained within boosting models compared to standalone `DecisionTreeRegressor` objects, but they should be very similar.

In [ ]:
# your code here

# 2. Learning rates

A crucial technique when boosting is that of *shrinkage* or *learning rate*. Given a trained model $f$, and a new model $h$ (the next boosting term) trained to predict the residual $y - f(X)$ (or to minimise a suitable cost function), the idea is to update $f$ by only a small multiple of $h$, $f + \varepsilon\cdot h$ (usually with $\varepsilon$ quite small, say $\varepsilon\approx 0.1$ or $\varepsilon\approx 0.05$) rather than to perform a full update $f + h$. This is necessary, since  boosting models can overfit otherwise- error on the training set is minimised so quickly and efficiently by the boosted model that performance on the testing set has almost no chance to improve. Applying shrinkage when constructed a boosted model will often mean that more boosting iterations are required, but model performance will usually be better as a result.
1. Add an extra boosting stage with shrinkage (use $\varepsilon = 0.5$) to the 'hand made' boosted models that you have already constructed. That is, take the model `lasso + dt_res` that you have already trained, train a new decision tree, say `dt2`, to predict `y - lasso(X) - dt_res(X)` and then examine the performance of the model `lasso(X) + dt_res(X) + 0.5 * dt2(X)`. Do the same for the other simple sum of decision trees model.

In [ ]:
# your code here

### Learning rates, overfitting, and early stopping

Boosted models are typically trained with hundreds or thousands of individual shallow decision trees (sometimes known as *decision stumps*). All common implementations of boosting will implement some form of shrinkage by default (both `sklearn` and `XGBoost`'s implementations default to $\varepsilon = 0.1$)- let's see what would happen if no shrinkage was present:

2. Train a `GradientBoostingRegressor` model with `max_depth=3, n_estimators=2_000` and `learning_rate=1` (so that no shrinkage is applied). Now use the method `.staged_predict` to compute the predictions made by the different stages of the model- plot a graph of the `rmse` of your model on `(X_train, y_train)` and `(X_test, y_test)`. What do you notice?
3. Now do the same but with the default setting `learning_rate=0.1`.

In [ ]:
# your code here

We've just constructed a relatively extreme example of overfitting, but it's possible to overfit more subtly without realising it.

4. Repeat step 2. with the settings `max_depth=5, n_estimators=2_000, learning_rate=0.1`. At which stage of the boosting process does your model achieve its minimum `rmse` on `X_test`? How does this compare to the `rmse` on `X_test` obtained by the final stage of the model?

In [ ]:
# your code here

The problem here is simple: we didn't know when to stop training!
* The concept of *early stopping* offers us a way around this- before training begins, we set say 20% of the training set aside to use as a validation set and we record the performance of our model on this validation set at each stage of the boosting process.
* Tracking validation performance like this lets us know if we're overfitting and, if a certain number of iterations pass without any improvement in the validation `rmse`, we know that it's a good idea to stop training early. 
* Tuning the learning rate of your boosting algorithm can be difficult- a good strategy is to optimise other hyperparameters (such as tree structure, regularisation) with `learning_rate=0.1` or `learning_rate=0.05` so long as performance is acceptable before then simultaneously decreasing the learning rate by a factor of `0.5` whilst increasing `n_estimators` to see if a performance improvement can be obtained.

# 3. XGBoost

We've been working with `sklearn`'s simple boosting implementation so far- let's now switch to using `XGBoost`'s implementation. `XGBoost` can be much faster than `sklearn`'s `GradientBoostingRegressor` (especially for deep trees), but it also has more convenient tooling for implementing early stopping. `XGBoost`'s `XGBRegressor` has an API that's very similar to that of `sklearn`. For instance, we can fit models which are very similar to the ones we've implemented so far as follows:

In [ ]:
# Specify the cost function/ objective manually in order to suppress a deprecation warning.
xgbr = XGBRegressor(
    objective="reg:squarederror",
    max_depth=3,
    n_estimators=2000,
    learning_rate=0.1,
    n_jobs=-1,
)
xgbr.fit(X_train, y_train)

Early stopping is implemented within `XGBoost` as follows:

In [ ]:
# Split our training data into a smaller training subset and a validation set.
X_tr, X_ev, y_tr, y_ev = train_test_split(X_train, y_train, train_size=0.8)
eval_set = [(X_tr, y_tr), (X_ev, y_ev)]
# eval_metric controls the loss function with respect to which the early stopping decision is made.
xgbr.fit(
    X_tr,
    y_tr,
    eval_set=eval_set,
    eval_metric="rmse",
    early_stopping_rounds=300,
    verbose=200,
)

`eval_metric` is set to the root mean squared error here, which is the same as the loss function that we're training to optimise. You could in principle use a different function for `eval_metric` than your training objective, but there is very rarely a good reason to do so. `verbose` controls how often metrics are printed to `stdout` by `.fit`.

* A good boosting strategy is to train models with `n_estimators` set to a very high value (say `50_000`) with early stopping (say, `early_stopping_rounds=300`) so that the boosting process effectively goes on forever until performance on the validation set has stopped improving.
* Note that, if you decrease the learning rate, you'll need to correspondingly increase `early_stopping_rounds`.
* You can find out which boosting iteration gave rise to the best performance by accessing the `.best_ntree_limit` attribute, and you can access the full evaluation history of `eval_metric` across `eval_set` using the `evals_result()` method.

1. Train three different `XGBoostRegressor` objects with the same parameters as above, using early stopping, but with different `train_test_split` random seeds. Record the different values of `model.best_ntree_limit`, and best validation error (you can obtain this using `model.evals_result()` and `model.best_iteration`) that you obtain. What do you notice?

In [ ]:
# your code here

* You should find that these values vary across different test train splits- this means that you'll need to use cross validation in order to be sure that your values for `.best_ntree_limit` are relatively stable.
* One downside of early stopping is that it means that we're not 'making full use' of our training data- we have to hold some of it out for validation. A solution to this problem is to find a good value for `.best_ntree_limit` and other hyper parameters using early stopping and cross validation before then training a new model on all of the training data with `n_estimators` set to the optimal value of `.best_ntree_limit`.
2. Train a new `XGBoostRegressor` model on all of `X_train` with `n_estimators` set to a value chosen by considering the `.best_ntree_limit` values that you obtained in step 1. Compare its performance on `X_test` to that of the models obtained in step 1.

In [ ]:
# your code here